# 02 - Data from the Web

## Assignment
1. Obtain the 200 top-ranking universities in www.topuniversities.com ([ranking 2018](https://www.topuniversities.com/university-rankings/world-university-rankings/2018)). In particular, extract the following fields for each university: name, rank, country and region, number of faculty members (international and total) and number of students (international and total). Some information is not available in the main list and you have to find them in the [details page](https://www.topuniversities.com/universities/ecole-polytechnique-fédérale-de-lausanne-epfl).
Store the resulting dataset in a pandas DataFrame and answer the following questions:
- Which are the best universities in term of: (a) ratio between faculty members and students, (b) ratio of international students?
- Answer the previous question aggregating the data by (c) country and (d) region.

Plot your data using bar charts and describe briefly what you observed.

2. Obtain the 200 top-ranking universities in www.timeshighereducation.com ([ranking 2018](http://timeshighereducation.com/world-university-rankings/2018/world-ranking)). Repeat the analysis of the previous point and discuss briefly what you observed.

3. Merge the two DataFrames created in questions 1 and 2 using university names. Match universities' names as well as you can, and explain your strategy. Keep track of the original position in both rankings.

4. Find useful insights in the data by performing an exploratory analysis. Can you find a strong correlation between any pair of variables in the dataset you just created? Example: when a university is strong in its international dimension, can you observe a consistency both for students and faculty members?

5. Can you find the best university taking in consideration both rankings? Explain your approach.

Hints:
- Keep your Notebook clean and don't print the verbose output of the requests if this does not add useful information for the reader.
- In case of tie, use the order defined in the webpage.


## 1. World University Rankings by TopUniversities

---

We will focus on data scraping from the [TopUniversities 2018 rankings](https://www.topuniversities.com/university-rankings/world-university-rankings/2018), as well as asnwering the assignment questions by performing simple data analysis. The structure of this chapter follows the assignment outline:

1. Data scraping
2. Ranking: faculty members, students and internationalization
3. Ranking: regional and national results


### Libraries utilized

We will use the BeautifulSoup along with Pandas and standard Python utilities for performing the scraping and data analysis.

In [1]:
# importing the utils for Web scraping
import requests
from bs4 import BeautifulSoup

# importing python and data utils
import pandas as pd
import numpy as np
import json
%matplotlib inline
import matplotlib.pyplot as plt

### Swiss army knife for data scraping

[Postman](https://www.getpostman.com/) is utilized outside Python to get more sense about the Web jungle we are trying to harvest our data from. We have used both Interceptor and the application itself to make sense of incoming traffic. 

We have also relied on traffic observations in Chrome Development Tools (*which can be accessed in Chrome browsers on Windows platform by pressing F12*).

### 1.1 Data scraping

The first step in our data scraping process was to get acquainted with the website itself. We have turned on the Interceptor for the traffic and the application itself, then loaded the website and observed the requests. 

#### Making our life easier: Postman filters

It is useful to filter out results of requests sent to the Website of interest. In modern websites it is common to have many 3rd party APIs which would make it somewhat more difficult to find the requests of interest. Additionally, trafic from other tabs in browser (if open) will appear, so it is additionally useful to display only what is the focus of our search. Such benefits are visible in the practical example bellow:

<table width="70%">
  <tr>
    <th style="text-align:center">Requests without any filters</th>
    <th style="text-align:center">After filtering results from *www.topuniversities.com* only</th> 
  </tr>
  <tr>
    <td>![](images/postman0.png)</td>
    <td style="vertical-align:top">![](images/postman1.png)</td> 
  </tr>
</table>

From the vast list of various requests, we have localized what seems to be important for the given task, and the requests for resources for the TopUniversity rankings. By analyzing the requests, we can see one **GET** request which obtains a textual file:

[https://www.topuniversities.com/sites/default/files/qs-rankings-data/357051.txt?_=1508597583828](https://www.topuniversities.com/sites/default/files/qs-rankings-data/357051.txt?_=1508597583828)

By analyzing the file, it turns out to be a very useful JSON! All the data needed for scraping at this level is present here, most notably the ranking and the URL towards more information. It correlates to the same information we can directly see from the Chrome Developer Tools:

![](images/chrome0.png)

#####  Design choice
As in the image above can be seen, the HTML data is well organized in the tables, with appropriate classes which designate the content of the cell. One approach would be to use `BeautifulSoup` and parse the HTML of the page. Since we have the data in more concise format (JSON) we decided to process the textual representation. Since only the representation and intermediate steps differ, and results are the same, this is a choice we have made. 

Nevertheless, further steps will have to rely on parsing the HTML and DOM of the subsequent university links with further details (since no equivalent JSON is obtained), so both approaches are showcased.


#### Implementing the scraping procedure

We will extract the before found JSON file and use it as a baseline for scraping this Website. We send a GET request and process the file as JSON.

In [2]:
## TU = TopUniversities
TU_path = 'https://www.topuniversities.com'
base_json_path = '/sites/default/files/qs-rankings-data/357051.txt?_=1508597583828'

req = requests.get(TU_path+base_json_path)

TU_json = req.json()

We convert the JSON file to Pandas `DataFrame` to enable easier exploration and utilization of the file. We are interested in the **data** section of our JSON file, and we take the first 200 top ranking universities in the list. 

Notably, some universities share the same ranking (tie). Per assignment at this point we will take 200 best ranking universities for our analysis, and not the universities until the rank 200.

In [6]:
TU_df = pd.DataFrame.from_dict(TU_json['data']).head(200)

We can observe that we did not need any additional parsing which would add both to code complexity, potential computational overhead in bigger datasets, as well as algorithmic complexity if the website HTML code is not tidy (which is not the case here). The `DataFrame` carries all the information we need for now, of which the region, country, rank, title and url play the significant role in the task.

In [7]:
TU_df.head(2)

,cc,core_id,country,guide,logo,nid,rank_display,region,score,stars,title,url
0,US,410,United States,"<a href=""/where-to-study/north-america/united-...","<img src=""https://www.topuniversities.com/site...",294850,1,North America,100,6,Massachusetts Institute of Technology (MIT),/universities/massachusetts-institute-technolo...
1,US,573,United States,"<a href=""/where-to-study/north-america/united-...","<img src=""https://www.topuniversities.com/site...",297282,2,North America,98.7,5,Stanford University,/universities/stanford-university


#### 1.1.1 Extracting the information for each University

Some information such as University name or rank is available immediately, while for detailed information we need to follow the subsequent link provided in the **url** field. Unfortunately, as previously mentioned, we don't have information packed in JSON format this time. We use HTML parsing provided by `BeautifulSoup` to obtain all the missing data. We are in luck since the HTML is well organized, with appropriate classes assigned to `div`s containing needed pieces of information.

We iterate through the previously obtained `DataFrame` and scrape and collect the necessary data, and append it to the existing `DataFrame`.

#### Note on undefined values

At least one of the universities has missing data and we need to handle such cases. If the number of such cases is small enough, we could impute the values manually, by consulting the appropriate sources, thus keeping the consistency of the rankings. If we don't have all the data for analysis, we would have to discard such Universities or come up with a methodology for mining up missing data on larger scale.

We choose to be consistent and we will print out the universities which have some missing values and will try to fill in the missing values.

In [8]:
# A helper function to get an number (integer) from the div text
def div2num(div):
    return int(div.find('div', class_='number').text.split()[0].replace(',',''))

In [9]:
for ind, row in TU_df.iterrows():
    # getting the detailed page of the given university
    # relative link to the page is given in the url field of the dataframe
    uni_details = requests.get(TU_path+row['url'])
    
    # get the HTML of the detailed university page and add it to the soup
    soup = BeautifulSoup(uni_details.text, 'html.parser')
    
    # get the name of the university from the DF
    uni_name = row['title']
    
    # get the number of faculty members from the detailed page and append to DF
    faculty_num = soup.find('div', class_='total faculty')
    try:
        # we parse the div text and append it to the DataFrame
        TU_df.loc[ind, 'Total faculty members'] = div2num(faculty_num)
    except:
        # print out the message in order to try to impute the data
        print('Undefined total faculty members for: '+uni_name)
    
    # similar procedure for other required data points
    students_num = soup.find('div', class_='total student')
    try:
        TU_df.loc[ind, 'Total students'] = div2num(students_num)
    except:
        print('Undefined total students for: '+uni_name)
    
    intl_faculty_num = soup.find('div', class_='inter faculty')
    try:
        TU_df.loc[ind, 'International faculty members'] = div2num(intl_faculty_num)
    except:
        print('Undefined international faculty members for: '+uni_name)
    
    intl_students_num = soup.find('div', class_='total inter')
    try:
        TU_df.loc[ind, 'International students'] = div2num(intl_students_num)
    except:
        print('Undefined international students for: '+uni_name)

Undefined total faculty members for: New York University (NYU)
Undefined total students for: New York University (NYU)
Undefined international faculty members for: New York University (NYU)
Undefined international students for: New York University (NYU)
Undefined international faculty members for: Indian Institute of Science (IISc) Bangalore


#### Imputing undefined values

We can see that the New York University (NYU) has information that is completely missing, while Indian Institute of Science (IISc) Bangalore has one missing value. Considering the scope of missing values, they will be manually set using the following sources:

- NYU: https://www.nyu.edu/about/news-publications/nyu-at-a-glance.html, https://www.nyu.edu/admissions/undergraduate-admissions/nyu-facts.html
- IISc: http://www.iisc.ac.in/iisc-in-numbers/#ffs-tabbed-12

This is potentially troubling, since there is no information about the actual data collection for the ranking, methodology or actual date which is available for a confident selection of the figures missing. Frequently, the websites also omit the precise dates. Comparing the potential noise which would be added to the fact that we would discard 2 top 200 universities, the decision is to take the data and impute it, while being aware that it might induce some error in methodology. It is a tradeoff between statistical error in imputed data and the error which would occur in the further aggregate analysis.

Therefore the NYU data is following:
- total faculty members: 7861 (Fall 2015)
- total students: 58419 (Fall 2015)
- international faculty members: 619 (September 2017)
- international students: 26% == 15189 (Fall 2015)

Unfortunately, the data of international faculty members is not available in public records online for IISc, therefore it will be designated as 0. The unavailability might rise from the fact that the figure is rather low (https://scroll.in/article/811696/there-are-no-indian-universities-in-the-worlds-top-250-list-heres-how-javadekar-can-change-that), so it would not be positive for faculty to publicize such information. 

*If time permitted a good practice would be to contact the institution directly for the actual quote in both cases*

In [10]:
TU_df.loc[TU_df.title=='New York University (NYU)','Total faculty members'] = 7861
TU_df.loc[TU_df.title=='New York University (NYU)','Total students'] = 58419
TU_df.loc[TU_df.title=='New York University (NYU)','International faculty members'] = 619
TU_df.loc[TU_df.title=='New York University (NYU)','International students'] = 15189


TU_df = TU_df.fillna(0)

#### Pruning the data in DataFrame

We have several pieces of information that are redundant at this point, so we will remove such columns from the DataFrame for a better overview. We can see bellow that *cc*, *rank_display*, *stars*, *core_id*, *guide*, *logo*, *nid* and *url* are present, but redundant in the following analysis.

In [11]:
TU_df.head(1)

,cc,core_id,country,guide,logo,nid,rank_display,region,score,stars,title,url,Total faculty members,Total students,International faculty members,International students
0,US,410,United States,"<a href=""/where-to-study/north-america/united-...","<img src=""https://www.topuniversities.com/site...",294850,1,North America,100,6,Massachusetts Institute of Technology (MIT),/universities/massachusetts-institute-technolo...,2982.0,11067.0,1679.0,3717.0


In [12]:
TU_df.drop(['cc', 'stars', 'core_id', 'guide', 'logo', 'nid', 'url'], axis=1, inplace=True)
TU_df.head(1)

,country,rank_display,region,score,title,Total faculty members,Total students,International faculty members,International students
0,United States,1,North America,100,Massachusetts Institute of Technology (MIT),2982.0,11067.0,1679.0,3717.0


#### We have our DataFrame ready for further analysis!
---

### 1.2 Ranking: faculty members, students and internationalization

In this section we answer the question: **Which are the best universities in term of: (a) ratio between faculty members and students, (b) ratio of international students?**

For easier analysis for (a) and (b) we will add the ratios to the existing DataFrame:

In [13]:
# ratio between faculty members and students
TU_df['Ratio faculty/students'] = TU_df['Total faculty members']/TU_df['Total students']

# ratio of international students
TU_df['Ratio intl/total students'] = TU_df['International students']/TU_df['Total students']

In [14]:
TU_df.head(1)

,country,rank_display,region,score,title,Total faculty members,Total students,International faculty members,International students,Ratio faculty/students,Ratio intl/total students
0,United States,1,North America,100,Massachusetts Institute of Technology (MIT),2982.0,11067.0,1679.0,3717.0,0.26945,0.335863


We have prepared the data for our analysis! The results are obtained by sorting the `DataFrame` by appropriate criterion.

### Best university: (a) ratio between faculty members and students

In [15]:
TU_fac_stud_df = TU_df.sort_values('Ratio faculty/students', ascending=False)

#### Top 5 by ratio of faculty members and students:

In [16]:
TU_fac_stud_df.head(5)

,country,rank_display,region,score,title,Total faculty members,Total students,International faculty members,International students,Ratio faculty/students,Ratio intl/total students
3,United States,4,North America,97.7,California Institute of Technology (Caltech),953.0,2255.0,350.0,647.0,0.422616,0.286918
15,United States,16,North America,90.4,Yale University,4940.0,12402.0,1708.0,2469.0,0.398323,0.199081
5,United Kingdom,6,Europe,95.3,University of Oxford,6750.0,19720.0,2964.0,7353.0,0.342292,0.372870
4,United Kingdom,5,Europe,95.6,University of Cambridge,5490.0,18770.0,2278.0,6699.0,0.292488,0.356899
16,United States,17,North America,89.8,Johns Hopkins University,4462.0,16146.0,1061.0,4105.0,0.276353,0.254243


### Best university: (b) ratio of international students

In [17]:
TU_intl_df = TU_df.sort_values('Ratio intl/total students', ascending=False)

#### Top 5 by ratio of international students

In [18]:
TU_intl_df.head(5)

,country,rank_display,region,score,title,Total faculty members,Total students,International faculty members,International students,Ratio faculty/students,Ratio intl/total students
34,United Kingdom,35,Europe,81.8,London School of Economics and Political Scien...,1088.0,9760.0,687.0,6748.0,0.111475,0.691393
11,Switzerland,12,Europe,91.2,Ecole Polytechnique Fédérale de Lausanne (EPFL),1695.0,10343.0,1300.0,5896.0,0.163879,0.570047
7,United Kingdom,8,Europe,93.7,Imperial College London,3930.0,16090.0,2071.0,8746.0,0.244251,0.543567
198,Netherlands,200,Europe,47.9,Maastricht University,1277.0,16385.0,502.0,8234.0,0.077937,0.502533
47,United States,=47,North America,78.6,Carnegie Mellon University,1342.0,13356.0,425.0,6385.0,0.100479,0.478062


------
### 1.3 Ranking: regional and national results

In this section we use the results obtained in **1.2** and aggregate them, separately, by country and region, thus providing answer to the question 2:

*Answer the previous question aggregating the data by (c) country and (d) region.*

The `DataFrame` contains all the datapoints we need for answering the questions, the only procedure we need to undertake is aggregation of data. We need to remove the calculated ratios per faculty since the summation would not result in proper result in this case (we would need to normalize the result otherwise). We can further remove the score and name of the university **manually**, since it does not play a role in the dataset, but since it is a variable on which we could not perform aggregation such variables would anyway be ignored by *Pandas*.

In [19]:
TU_aggregate_df = TU_df.copy()
TU_aggregate_df.drop(['Ratio faculty/students', 'Ratio intl/total students'], axis=1, inplace=True)
TU_aggregate_df.head(1)

,country,rank_display,region,score,title,Total faculty members,Total students,International faculty members,International students
0,United States,1,North America,100,Massachusetts Institute of Technology (MIT),2982.0,11067.0,1679.0,3717.0


---
#### Aggregating data by country (c)

We will group the data by the country and then sum the values of such aggregate to produce desired data.

In [20]:
# we can compose multiple functions to reduce code clutter
TU_country_df = TU_aggregate_df.groupby(['country']).sum()
TU_country_df.head(2)

,Total faculty members,Total students,International faculty members,International students
country,,,,
Argentina,16421.0,122301.0,3165.0,27109.0
Australia,22034.0,301994.0,11382.0,106359.0


As done previously, desired ratios are calculated and added to the `DataFrame`:

In [21]:
TU_country_df['Ratio faculty/students'] = TU_country_df['Total faculty members']/TU_country_df['Total students']
TU_country_df['Ratio intl/total students'] = TU_country_df['International students']/TU_country_df['Total students']

#### Top 5 countries with Universities with highest ratio of faculty to students

In [22]:
TU_country_df.sort_values('Ratio faculty/students', ascending=False).head(5)

,Total faculty members,Total students,International faculty members,International students,Ratio faculty/students,Ratio intl/total students
country,,,,,,
Russia,6709.0,30233.0,373.0,5098.0,0.221910,0.168624
Denmark,11916.0,67223.0,3904.0,9543.0,0.177261,0.141960
Saudi Arabia,1062.0,6040.0,665.0,989.0,0.175828,0.163742
Singapore,9444.0,58466.0,6079.0,16168.0,0.161530,0.276537
Malaysia,2755.0,17902.0,655.0,3476.0,0.153893,0.194168


#### Top 5 countries with Universities with highest ratio of international students

In [23]:
TU_country_df.sort_values('Ratio intl/total students', ascending=False).head(5)

,Total faculty members,Total students,International faculty members,International students,Ratio faculty/students,Ratio intl/total students
country,,,,,,
Australia,22034.0,301994.0,11382.0,106359.0,0.072962,0.352189
United Kingdom,79934.0,583621.0,30216.0,199426.0,0.136962,0.341705
Hong Kong,10166.0,78838.0,6296.0,24499.0,0.128948,0.310751
Austria,4117.0,63446.0,1572.0,19667.0,0.064890,0.309980
Switzerland,15323.0,109112.0,9208.0,32995.0,0.140434,0.302396


---
#### Aggregating data by region (d)

We perform the same procedure with grouping by the region instead of country.

In [24]:
TU_region_df = TU_aggregate_df.groupby(['region']).sum()
TU_region_df.head(2)

,Total faculty members,Total students,International faculty members,International students
region,,,,
Africa,1733.0,19593.0,379.0,3325.0
Asia,106734.0,807003.0,25462.0,110100.0


We calculate the desired ratios by the region and then present the top 5 regions by each criterion.

In [25]:
TU_region_df['Ratio faculty/students'] = TU_region_df['Total faculty members']/TU_region_df['Total students']
TU_region_df['Ratio intl/total students'] = TU_region_df['International students']/TU_region_df['Total students']

#### Top 5 regions with Universities with the highest faculty to student ratio

In [26]:
TU_region_df.sort_values(['Ratio faculty/students'], ascending=False).head(5)

,Total faculty members,Total students,International faculty members,International students,Ratio faculty/students,Ratio intl/total students
region,,,,,,
Asia,106734.0,807003.0,25462.0,110100.0,0.132260,0.136431
North America,189984.0,1604772.0,44455.0,307305.0,0.118387,0.191494
Europe,218358.0,1957251.0,67598.0,449364.0,0.111564,0.229589
Latin America,45382.0,435750.0,5648.0,36871.0,0.104147,0.084615
Africa,1733.0,19593.0,379.0,3325.0,0.088450,0.169703


#### Top 5 regions with Universities with the highest ratio of international students

In [27]:
TU_region_df.sort_values(['Ratio intl/total students'], ascending=False).head(5)

,Total faculty members,Total students,International faculty members,International students,Ratio faculty/students,Ratio intl/total students
region,,,,,,
Oceania,25347.0,350167.0,12786.0,118798.0,0.072385,0.339261
Europe,218358.0,1957251.0,67598.0,449364.0,0.111564,0.229589
North America,189984.0,1604772.0,44455.0,307305.0,0.118387,0.191494
Africa,1733.0,19593.0,379.0,3325.0,0.088450,0.169703
Asia,106734.0,807003.0,25462.0,110100.0,0.132260,0.136431
